In [5]:
#### Importing necessary libraries
import pandas as pd
import nltk
import glob
import os
import re
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!unzip '/content/drive/MyDrive/BBC News Train.csv.zip'

Archive:  /content/drive/MyDrive/BBC News Train.csv.zip
  inflating: BBC News Train.csv      


In [8]:
df=pd.read_csv("/content/BBC News Train.csv",encoding='latin1')

In [9]:
df

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


# Pre-Processing 


In [10]:
### Remove stopwords
def rem_sw(arr):
  global stop_words
  reg_str = r'^'
  for i in range(len(arr)):
    for sw in stop_words:
      reg_str+=sw
      reg_str+='$'
      arr[i] = re.sub(reg_str, '', arr[i])
      reg_str = r'^'
  return arr

# Remove punctuations
def rem_punc(str):
  str = re.sub(r'(\.|\?|\,|!|\:|\;|\&|\-|\(|\)|\{|\}|\'|\"|\/)', ' ', str)
  return str


lemmatizer = WordNetLemmatizer()

In [11]:
for i in range(len(df['Text'])):
  df['Text'][i]=df['Text'][i].lower()                                           ### Lowercasing the text
  df['Text'][i] = rem_punc(df['Text'][i])                                       ### Removing punctuations
  df['Text'][i] = word_tokenize(df['Text'][i])                                  ### Tokenising
  df['Text'][i] = rem_sw(df['Text'][i])                                         ### Removing stopwords
  for j in range(len(df['Text'][i])):
      df['Text'][i][j]=lemmatizer.lemmatize(df['Text'][i][j])                   ### Lemmatising



<ipython-input-11-9365b2e12651>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'][i]=df['Text'][i].lower()                                           ### Lowercasing the text
<ipython-input-11-9365b2e12651>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'][i] = rem_punc(df['Text'][i])                                       ### Removing punctuations
<ipython-input-11-9365b2e12651>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [12]:
### Dropping extra columns
df=df.drop('ArticleId',axis=1)

In [13]:
df

,Text,Category
0,"[worldcom, ex, bos, launch, defence, lawyer, d...",business
1,"[german, business, confidence, slide, german, ...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, boss, , $, 168m, payout, eighteen, for...",business
...,...,...
1485,"[double, eviction, , big, brother, model, capr...",entertainment
1486,"[dj, double, act, revamp, chart, show, dj, duo...",entertainment
1487,"[weak, dollar, hit, reuters, revenue, , medium...",business
1488,"[apple, ipod, family, expands, market, apple, ...",tech


In [14]:
for i in range(len(df['Text'])):
  str=""
  for s in df['Text'][i]:
    str+=s+" "
  df['Text'][i]=str


# train-test split=0.3

TF-ICF


In [15]:
def tf_icf(X, y, vocab):
    n = X.shape[0]

    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
    
    
    
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
    
    
    tf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_icf[i,j] = tf[i,j]  * icf[j]
    
    
    return tf_icf

In [16]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf = tf_icf(df['Text'], df['Category'], vocab)



In [17]:
len(vocab)

22278

In [75]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf, df['Category'], test_size=0.3)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

Training Performance


In [76]:
y_pred = nb_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Training Accuracy:", accuracy)


Training Accuracy: 0.9952061361457335


In [20]:
## Probability of each category in training set
classes={'sport':0,'entertainment':0,'politics':0,'business':0,'tech':0}
for i in y_train:
  classes[i]+=1

for i in classes:
  classes[i]/=len(y_train)

classes

{'sport': 0.22722914669223393,
 'entertainment': 0.18120805369127516,
 'politics': 0.19367209971236818,
 'business': 0.22722914669223393,
 'tech': 0.17066155321188878}

In [21]:
## Probability if each feature belonging to each category
feature_probabilities = {}
for category_index, category in enumerate(nb_classifier.classes_):
    feature_probabilities[category] = {}
    category_features = X_train[y_train == category]
  
    category_feature_count = category_features.sum(axis=0)
  
    for i in range(X_train.shape[0]):
        if(y_train.iloc[i]==category):
            for j in range(X_train.shape[1]):
                feature_probabilities[category][j]=category_feature_count[j] / category_feature_count.sum()
    


In [22]:
feature_probabilities

{'business': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 6.0232497440118856e-05,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 2.0077499146706285e-05,
  9: 4.015499829341257e-05,
  10: 4.015499829341257e-05,
  11: 0.0,
  12: 0.00038147248378741946,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 2.0077499146706285e-05,
  19: 2.0077499146706285e-05,
  20: 2.0077499146706285e-05,
  21: 2.0077499146706285e-05,
  22: 0.0,
  23: 0.0,
  24: 2.0077499146706285e-05,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: 0.0,
  35: 4.015499829341257e-05,
  36: 0.0,
  37: 0.0,
  38: 0.0,
  39: 0.0,
  40: 0.0,
  41: 0.000140542494026944,
  42: 0.0,
  43: 2.0077499146706285e-05,
  44: 4.015499829341257e-05,
  45: 0.0,
  46: 0.0,
  47: 0.0,
  48: 0.0,
  49: 0.0,
  50: 0.0,
  51: 0.00012046499488023771,
  52: 0.0,
  53: 0.00010038749573353143,
  54: 2.0077499146706285e-05,
  55: 6.0232497440118856e-05,
  56: 0.0,
  57: 0.0,
  58: 0.0003613949846407

Testing Performance


In [77]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9776286353467561
Precision: 0.9789594831811761
Recall: 0.9776286353467561
F1-score: 0.9775415168958264


In [24]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.98      0.95      0.96        99
entertainment       0.96      0.96      0.96        84
     politics       0.92      0.97      0.95        72
        sport       1.00      1.00      1.00       109
         tech       0.96      0.95      0.96        83

     accuracy                           0.97       447
    macro avg       0.97      0.97      0.97       447
 weighted avg       0.97      0.97      0.97       447



In [25]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat


array([[ 94,   0,   4,   0,   1],
       [  1,  81,   1,   0,   1],
       [  1,   0,  70,   0,   1],
       [  0,   0,   0, 109,   0],
       [  0,   3,   1,   0,  79]])

Testing with diff parameters

In [78]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.912751677852349
Precision: 0.9142655359359462
Recall: 0.912751677852349
F1-score: 0.9123045257711311


In [83]:
## with alpha=0.6
nb_classifier = MultinomialNB(alpha=0.6)
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9798657718120806
Precision: 0.9809966841763424
Recall: 0.9798657718120806
F1-score: 0.9797479205174963


Tf-IDF-ICF

In [28]:
def tf_icf_idf(X, y, vocab):
    n = X.shape[0]
   
    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
   
   
    idf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        df = 0
        for i in range(n):
            if tf[i,j] > 0:
                df += 1
        idf[j] = np.log(n/df)
   
   
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
   
   
    tf_idf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_idf_icf[i,j] = tf[i,j] * idf[j] * icf[j]
   
   
    return tf_idf_icf

In [29]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf_idf = tf_icf_idf(df['Text'], df['Category'], vocab)

In [30]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf_idf, df['Category'], test_size=0.3,random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9798657718120806
Precision: 0.9800668875497954
Recall: 0.9798657718120806
F1-score: 0.9799166342168256


TF-IDF


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['Text'])
X_train, X_test, y_train, y_test = train_test_split(tfidf, df['Category'], test_size=0.3, random_state=42)

nb=MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)




Testing Accuracy: 0.970917225950783
Precision: 0.9712988371053634
Recall: 0.970917225950783
F1-score: 0.9708945788234244


In [32]:
feature_probabilities = {}
for category_index, category in enumerate(nb_classifier.classes_):
    feature_probabilities[category] = {}
    category_features = X_train[y_train == category]
    category_feature_count = category_features.sum(axis=0)
    category_feature_count = category_feature_count.A1
    for feature_index, feature in enumerate(tfidf_vectorizer.get_feature_names_out()):
        feature_probabilities[category][feature] = category_feature_count[feature_index] / category_feature_count.sum()

feature_probabilities

{'business': {'00': 0.0,
  '000': 0.0013337987107929796,
  '0001': 0.0,
  '000bn': 5.6455986828239134e-05,
  '000m': 0.0,
  '000th': 0.0,
  '001': 0.0,
  '001and': 0.0,
  '001st': 0.0,
  '0051': 0.0,
  '007': 0.0,
  '01': 3.537510368477473e-05,
  '0100': 0.0,
  '0130': 0.0,
  '028': 0.0,
  '03': 3.3603834103072795e-05,
  '033': 0.0,
  '04': 0.0,
  '0400': 0.0,
  '041': 0.0,
  '04bn': 0.0,
  '04secs': 0.0,
  '05': 3.99386412145099e-05,
  '050': 0.0,
  '050505': 0.0,
  '0530': 0.0,
  '0530gmt': 0.0,
  '056': 0.0,
  '05bn': 5.563278409058719e-05,
  '05m': 0.0,
  '06': 4.1783546870673525e-05,
  '060': 0.0,
  '0630': 0.0,
  '069': 4.978599445001278e-05,
  '07': 3.772719015569976e-05,
  '070': 0.0,
  '0700': 0.0,
  '0710': 4.190472231340872e-05,
  '072': 0.0,
  '0730': 0.0,
  '08': 4.590588280852135e-05,
  '080': 0.0,
  '0800': 0.0,
  '0845': 0.0,
  '0870': 0.0,
  '088': 6.686324520645529e-05,
  '09': 0.0,
  '0900': 0.0,
  '093': 0.0,
  '0950': 4.439979154770904e-05,
  '09bn': 0.0,
  '09secs

# train-test split=0.2

TF-ICF


In [33]:
def tf_icf(X, y, vocab):
    n = X.shape[0]

    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
    
    
    
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
    
    
    tf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_icf[i,j] = tf[i,j]  * icf[j]
    
    
    return tf_icf

In [34]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf = tf_icf(df['Text'], df['Category'], vocab)



In [35]:
len(vocab)

22278

In [36]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf, df['Category'], test_size=0.2)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

Training Performance


In [37]:
y_pred = nb_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Training Accuracy:", accuracy)


Training Accuracy: 0.9949664429530202


In [38]:
## Probability of each category in training set
classes={'sport':0,'entertainment':0,'politics':0,'business':0,'tech':0}
for i in y_train:
  classes[i]+=1

for i in classes:
  classes[i]/=len(y_train)

classes

{'sport': 0.22986577181208054,
 'entertainment': 0.18288590604026847,
 'politics': 0.18708053691275167,
 'business': 0.23238255033557048,
 'tech': 0.16778523489932887}

Testing Performance


In [39]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9865771812080537
Precision: 0.9867902418237988
Recall: 0.9865771812080537
F1-score: 0.9865691627977836


In [40]:
### with alpha=0.6
nb_classifier = MultinomialNB(alpha=0.6)
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9865771812080537
Precision: 0.9867902418237988
Recall: 0.9865771812080537
F1-score: 0.9865691627977836


In [41]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.98      0.98      0.98        59
entertainment       1.00      0.96      0.98        55
     politics       0.98      0.98      0.98        51
        sport       1.00      1.00      1.00        72
         tech       0.97      1.00      0.98        61

     accuracy                           0.99       298
    macro avg       0.99      0.99      0.99       298
 weighted avg       0.99      0.99      0.99       298



In [42]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat


array([[58,  0,  0,  0,  1],
       [ 0, 53,  1,  0,  1],
       [ 1,  0, 50,  0,  0],
       [ 0,  0,  0, 72,  0],
       [ 0,  0,  0,  0, 61]])

Tf-IDF-ICF

In [43]:
def tf_icf_idf(X, y, vocab):
    n = X.shape[0]
   
    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
   
   
    idf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        df = 0
        for i in range(n):
            if tf[i,j] > 0:
                df += 1
        idf[j] = np.log(n/df)
   
   
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
   
   
    tf_idf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_idf_icf[i,j] = tf[i,j] * idf[j] * icf[j]
   
   
    return tf_idf_icf

In [44]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf_idf = tf_icf_idf(df['Text'], df['Category'], vocab)

In [45]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf_idf, df['Category'], test_size=0.2,random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9697986577181208
Precision: 0.9703182829789123
Recall: 0.9697986577181208
F1-score: 0.9699168730271868


TF-IDF


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['Text'])
X_train, X_test, y_train, y_test = train_test_split(tfidf, df['Category'], test_size=0.2, random_state=42)

nb=MultinomialNB
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)




Testing Accuracy: 0.9563758389261745
Precision: 0.9574896655178124
Recall: 0.9563758389261745
F1-score: 0.9562904295627416


# train-test split=0.4

TF-ICF


In [47]:
def tf_icf(X, y, vocab):
    n = X.shape[0]

    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
    
    
    
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
    
    
    tf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_icf[i,j] = tf[i,j]  * icf[j]
    
    
    return tf_icf

In [48]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf = tf_icf(df['Text'], df['Category'], vocab)



In [49]:
len(vocab)

22278

In [50]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf, df['Category'], test_size=0.4)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

Training Performance


In [51]:
y_pred = nb_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Training Accuracy:", accuracy)


Training Accuracy: 0.9955257270693513


In [52]:
## Probability of each category in training set
classes={'sport':0,'entertainment':0,'politics':0,'business':0,'tech':0}
for i in y_train:
  classes[i]+=1

for i in classes:
  classes[i]/=len(y_train)

classes

{'sport': 0.2348993288590604,
 'entertainment': 0.18232662192393737,
 'politics': 0.16554809843400448,
 'business': 0.22595078299776286,
 'tech': 0.1912751677852349}

Testing Performance


In [53]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9781879194630873
Precision: 0.9785968828485272
Recall: 0.9781879194630873
F1-score: 0.9782360124933803


In [54]:
### with alpha=0.6
nb_classifier = MultinomialNB(alpha=0.6)
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9798657718120806
Precision: 0.980151516116297
Recall: 0.9798657718120806
F1-score: 0.9799095219110198


In [55]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.96      0.98      0.97       134
entertainment       1.00      0.97      0.99       110
     politics       0.97      0.97      0.97       126
        sport       1.00      0.99      0.99       136
         tech       0.97      1.00      0.98        90

     accuracy                           0.98       596
    macro avg       0.98      0.98      0.98       596
 weighted avg       0.98      0.98      0.98       596



In [56]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat


array([[131,   0,   1,   0,   2],
       [  0, 107,   2,   0,   1],
       [  4,   0, 122,   0,   0],
       [  1,   0,   1, 134,   0],
       [  0,   0,   0,   0,  90]])

Tf-IDF-ICF

In [57]:
def tf_icf_idf(X, y, vocab):
    n = X.shape[0]
   
    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
   
   
    idf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        df = 0
        for i in range(n):
            if tf[i,j] > 0:
                df += 1
        idf[j] = np.log(n/df)
   
   
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
   
   
    tf_idf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_idf_icf[i,j] = tf[i,j] * idf[j] * icf[j]
   
   
    return tf_idf_icf

In [58]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf_idf = tf_icf_idf(df['Text'], df['Category'], vocab)

In [59]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf_idf, df['Category'], test_size=0.4 , random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9748322147651006
Precision: 0.9752979962737479
Recall: 0.9748322147651006
F1-score: 0.974935465749418


TF-IDF


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['Text'])
X_train, X_test, y_train, y_test = train_test_split(tfidf, df['Category'], test_size=0.4 , random_state=42)

nb=MultinomialNB
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)




Testing Accuracy: 0.959731543624161
Precision: 0.9609130088463046
Recall: 0.959731543624161
F1-score: 0.9595975614739068


# train-test split=0.5

TF-ICF


In [61]:
def tf_icf(X, y, vocab):
    n = X.shape[0]

    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
    
    
    
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
    
    
    tf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_icf[i,j] = tf[i,j]  * icf[j]
    
    
    return tf_icf

In [62]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf = tf_icf(df['Text'], df['Category'], vocab)



In [63]:
len(vocab)

22278

In [64]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf, df['Category'], test_size=0.5)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

Training Performance


In [65]:
y_pred = nb_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Training Accuracy:", accuracy)


Training Accuracy: 0.9959731543624161


In [66]:
## Probability of each category in training set
classes={'sport':0,'entertainment':0,'politics':0,'business':0,'tech':0}
for i in y_train:
  classes[i]+=1

for i in classes:
  classes[i]/=len(y_train)

classes

{'sport': 0.24563758389261744,
 'entertainment': 0.17852348993288591,
 'politics': 0.17315436241610738,
 'business': 0.22550335570469798,
 'tech': 0.17718120805369128}

Testing Performance


In [67]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.9771812080536912
Precision: 0.9779968246356023
Recall: 0.9771812080536912
F1-score: 0.9772372749337948


In [68]:
### with alpha=0.6
nb_classifier = MultinomialNB(alpha=0.6)
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.978523489932886
Precision: 0.9792307725886684
Recall: 0.978523489932886
F1-score: 0.9785820230837359


In [69]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.99      0.95      0.97       168
entertainment       0.99      0.97      0.98       140
     politics       0.95      0.99      0.97       145
        sport       1.00      0.99      0.99       163
         tech       0.96      0.99      0.97       129

     accuracy                           0.98       745
    macro avg       0.98      0.98      0.98       745
 weighted avg       0.98      0.98      0.98       745



In [70]:
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat


array([[160,   0,   3,   0,   5],
       [  0, 136,   3,   0,   1],
       [  1,   0, 144,   0,   0],
       [  0,   1,   1, 161,   0],
       [  0,   0,   1,   0, 128]])

Tf-IDF-ICF

In [71]:
def tf_icf_idf(X, y, vocab):
    n = X.shape[0]
   
    tf = np.zeros((n, len(vocab)))
    for i in range(n):
        words = X.iloc[i].split()
        for w in words:
            if w in vocab:
                j = vocab.index(w)
                tf[i,j] += 1
   
   
    idf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        df = 0
        for i in range(n):
            if tf[i,j] > 0:
                df += 1
        idf[j] = np.log(n/df)
   
   
    icf = np.zeros(len(vocab))
    for j in range(len(vocab)):
        n_c = 0
        for i in range(n):
            if tf[i,j] > 0 and y.iloc[i] == 1:
                n_c += 1
        if(n_c==0):
          icf[j]=1
        else:
          icf[j] = np.log((n-n_c)/n_c)
   
   
    tf_idf_icf = np.zeros((n, len(vocab)))
    for i in range(n):
        for j in range(len(vocab)):
            tf_idf_icf[i,j] = tf[i,j] * idf[j] * icf[j]
   
   
    return tf_idf_icf

In [72]:
vocab = list(set(" ".join(df['Text']).split()))
tf_icf_idf = tf_icf_idf(df['Text'], df['Category'], vocab)

In [73]:
X_train,X_test,y_train,y_test=train_test_split(tf_icf_idf, df['Category'], test_size=0.5 , random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)


Testing Accuracy: 0.978523489932886
Precision: 0.9789113270053686
Recall: 0.978523489932886
F1-score: 0.9785884254017985


TF-IDF


In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['Text'])
X_train, X_test, y_train, y_test = train_test_split(tfidf, df['Category'], test_size=0.5 , random_state=42)

nb=MultinomialNB
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1score = f1_score(y_test, y_pred, average="weighted")

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)




Testing Accuracy: 0.9624161073825503
Precision: 0.963649980846942
Recall: 0.9624161073825503
F1-score: 0.9623685303830178
